In [1]:
import os 
import warnings 
warnings.filterwarnings('ignore')
from glob import glob 
from tqdm import tqdm 
import pickle 
import time 

import pandas as pd 
import numpy as np 
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score,roc_curve,auc

import torch 
import torch.nn as nn  
from torch import Tensor
from torch.utils.data import Dataset,DataLoader 
from torch import optim 

from src.model import Discriminator,Generator,weights_init
from src.Dataset import SwatDataset
from src.Options import OPT

In [2]:
with open('./Data/pre_data.pickle','rb') as f:
    data = pickle.load(f)
opt = OPT()

train_set = SwatDataset(data,'train')
test_set = SwatDataset(data,'test')
train_loader = DataLoader(train_set,batch_size=opt.batchsize,shuffle=True)
test_loader = DataLoader(test_set,batch_size=opt.batchsize,shuffle=False)

In [11]:
def update_netd(batch_x):
    D.zero_grad()
    # Train with real 
    out_d_real, feat_real = D(batch_x)

    # Train with fake 
    fake, latent_i = G(batch_x)
    out_d_fake, feat_fake = D(fake)

    err_d_real =  bce_criterion(
                                out_d_real.type(torch.float64), 
                                torch.full((opt.batchsize,),  real_label).type(torch.float64).to(device)
                                )
    err_d_fake =  bce_criterion(
                                out_d_fake.type(torch.float64), 
                                torch.full((opt.batchsize,),  fake_label).type(torch.float64).to(device)
                                )

    err_d = err_d_real + err_d_fake
    err_d.backward()
    optimizerD.step()
    return err_d_real,err_d_fake

def update_netg(batch_x):
    G.zero_grad()
    fake,latent_i = G(batch_x)
    out_g,feat_fake = D(fake)
    _,feat_real = D(batch_x)

    err_g_adv = mse_criterion(feat_fake,feat_real)
    err_g_rec = mse_criterion(fake,batch_x)

    err_g = err_g_rec + err_g_adv * opt.w_adv
    err_g.backward()
    optimizerG.step()
    return err_g_rec,err_g_adv
    
def reinitialize_netd():
        """ Initialize the weights of netD
        """
        D.apply(weights_init)
        print('Reloading d net')       
        
def optimize(batch_x):
    err_d_real,err_d_fake = update_netd(batch_x)
    err_g_rec,err_g_adv = update_netg(batch_x)
    
    err_d = err_d_real.item() + err_d_fake.item() 
    err_g = err_g_rec.item() + err_g_adv.item()
    
    errors = {'err_d':err_d,
                    'err_g': err_g,
                    'err_d_real': err_d_real.item(),
                    'err_d_fake': err_d_fake.item(),
                    'err_g_adv': err_g_adv.item(),
                    'err_g_rec': err_g_rec.item(),
                  }
    
    if err_d < 5e-6:
        reinitialize_netd()
    return  errors           

def train_epoch(dataloader):
    global total_steps
    G.train()
    D.train()
    epoch_iter = 0 
    for batch_x,batch_y in dataloader:
        total_steps += opt.batchsize 
        epoch_iter +=1 
        
        batch_x,batch_y = batch_x.type(torch.float32).to(device),batch_y.type(torch.float32).to(device)
        
        errors = optimize(batch_x)
        
        if (epoch_iter % 1000) == 0:
            print(f"\n Epoch : {cur_epoch} | [{epoch_iter}/{dataloader.__len__()}]" )
            print(f"\n D_loss(R/F) : {errors['err_d_real']:.4f}/{errors['err_d_fake']:.4f}, G_loss : {errors['err_g']:.4f}")

    
##predict 
def predict(dataloader,scale=True):
    G.eval()
    D.eval()
    with torch.no_grad():

        an_scores = torch.zeros(size=(len(dataloader.dataset),), dtype=torch.float32, device= device)
        gt_labels = torch.zeros(size=(len(dataloader.dataset),), dtype=torch.long,    device= device)
        dis_feat = torch.zeros(size=(len(dataloader.dataset),  opt.ndf*16*10), dtype=torch.float32,device= device)


        for i, (batch_x,batch_y) in enumerate(dataloader):
            batch_x,batch_y = batch_x.type(torch.float32).to(device),batch_y.type(torch.float32).to(device)
            fake, latent_i =  G(batch_x)
            # error = torch.mean(torch.pow((d_feat.view( input.shape[0],-1)-d_gen_feat.view( input.shape[0],-1)), 2), dim=1)
            #
            error = torch.mean(
                torch.pow(( batch_x.view( batch_x.shape[0], -1) -  fake.view( fake.shape[0], -1)), 2),
                dim=1) # reconstruction loss 


            gt_labels[i* opt.batchsize : i* opt.batchsize+error.size(0)] =  torch.max(batch_y,dim=1).values.reshape(error.size(0)) #y 
            an_scores[i* opt.batchsize : i* opt.batchsize+error.size(0)] = error.reshape(error.size(0)) #y_pred
            
        # Scale error vector between [0, 1]
        if scale:
             an_scores = (an_scores - torch.min(an_scores)) / (torch.max(an_scores) - torch.min(an_scores))

        y_= gt_labels.detach().cpu().numpy()
        y_pred = an_scores.detach().cpu().numpy()

        return y_,y_pred
#def evaluate(train_loader,test_loader):
def evaluate(test_loader):
    test_y,test_y_pred = predict(test_loader)
    train_y,train_y_pred = predict(train_loader)
    #thres = np.percentile(np.concatenate([test_y_pred,train_y_pred]),1)
    thres = 0.02 
    
    y_pred_thres = pd.Series(test_y_pred).apply(lambda x : 1 if x > thres else 0).values
    #auc = accuracy_score(y,y_pred)
    precision, recall, f_score, support = precision_recall_fscore_support(test_y, y_pred_thres,
                                                                                average='binary')
    fpr,tpr,thr = roc_curve(test_y,test_y_pred)
    auroc = auc(fpr,tpr)
    
    return precision, recall, f_score, auroc    

def validate(dataloader,thres):
    y_,y_pred = predict(dataloader)
    precision, recall, f_score, auroc = evaluate(y_,y_pred,thres)
    return precision, recall, f_score, auroc 
        

In [12]:
device = 'cuda:0'
G = Generator(opt).to(device)
G.apply(weights_init)

D = Discriminator(opt).to(device)
D.apply(weights_init)

bce_criterion = nn.BCELoss()
mse_criterion = nn.MSELoss()

optimizerD = optim.Adam(D.parameters(),lr=opt.lr,betas=(opt.beta1,0.999))
optimizerG = optim.Adam(G.parameters(),lr=opt.lr,betas=(opt.beta1,0.999))

total_steps = 0 
cur_epoch = 0 

real_label = 1 
fake_label = 0 

train_hist = {} 
train_hist['D_loss'] = []
train_hist['G_loss'] = []
best_f = 0 
best_f_epoch = 0 

for epoch in tqdm(range(opt.niter)):
    try:
        cur_epoch +=1 
        train_epoch(train_loader)
    except:
        pass                 
    precision, recall, f_score, auroc = evaluate(test_loader)
    print('\n ----------------------------------------------------------------')
    print(f'\n Precision : {precision:.3f} | Recall : {recall:.3f} | F1-score : {f_score:.3f} | AUROC : {auroc:.3f}')
    print('\n ----------------------------------------------------------------')
    
    
    if f_score > best_f:
        best_f = f_score
        best_f_epoch=cur_epoch
        
        torch.save(G,'./save_models/G.pt')
        torch.save(D,'./save_models/D.pt')


  0%|          | 0/100 [00:00<?, ?it/s]


 Epoch : 1 | [1000/2327]

 D_loss(R/F) : 0.0075/0.0638, G_loss : 0.3229

 Epoch : 1 | [2000/2327]

 D_loss(R/F) : 0.0254/0.0739, G_loss : 0.2329


  1%|          | 1/100 [00:59<1:38:21, 59.61s/it]


 ----------------------------------------------------------------

 Precision : 0.259 | Recall : 0.970 | F1-score : 0.408 | AUROC : 0.682

 ----------------------------------------------------------------

 Epoch : 2 | [1000/2327]

 D_loss(R/F) : 0.0083/0.0389, G_loss : 0.2328

 Epoch : 2 | [2000/2327]

 D_loss(R/F) : 0.0853/0.0434, G_loss : 0.1717


  2%|▏         | 2/100 [01:59<1:37:28, 59.68s/it]


 ----------------------------------------------------------------

 Precision : 0.285 | Recall : 0.929 | F1-score : 0.437 | AUROC : 0.687

 ----------------------------------------------------------------

 Epoch : 3 | [1000/2327]

 D_loss(R/F) : 0.0031/0.0468, G_loss : 0.1932

 Epoch : 3 | [2000/2327]

 D_loss(R/F) : 0.4446/0.0046, G_loss : 0.1617


  3%|▎         | 3/100 [02:59<1:36:36, 59.75s/it]


 ----------------------------------------------------------------

 Precision : 0.278 | Recall : 0.940 | F1-score : 0.429 | AUROC : 0.689

 ----------------------------------------------------------------

 Epoch : 4 | [1000/2327]

 D_loss(R/F) : 0.0021/0.1145, G_loss : 0.1727

 Epoch : 4 | [2000/2327]

 D_loss(R/F) : 0.0096/0.1177, G_loss : 0.1634


  4%|▍         | 4/100 [03:59<1:35:44, 59.84s/it]


 ----------------------------------------------------------------

 Precision : 0.280 | Recall : 0.933 | F1-score : 0.430 | AUROC : 0.688

 ----------------------------------------------------------------

 Epoch : 5 | [1000/2327]

 D_loss(R/F) : 0.0456/0.0114, G_loss : 0.1902
